In [1]:
from bs4 import BeautifulSoup
import urllib.request
import requests
from pprint import pprint
import pandas as pd
import numpy as np
import json
import copy
from datetime import datetime, timezone
from google.cloud import storage
import gcsfs
import re
from sqlalchemy import create_engine, text
from typing import List

In [2]:
postgres_config = {
    "host": "findy-medium-stage.czmgcqkw4ett.ap-southeast-1.rds.amazonaws.com",
    "database": "findy_medium_stage",
    "user": "postgres",
    "password": "F!nDy!Med!umStage2o24",
    "port": "5432"
}

In [3]:
def build_upsert_query(cols: List[str],
                       table_name: str,
                       unique_key: List[str]=[],
                       cols_not_for_update: List[str] = None) -> str:
    """
    Builds postgres upsert query using input arguments.
    Note: In the absence of unique_key, this will be just an insert query.
    Example : build_upsert_query(
        ['col1', 'col2', 'col3', 'col4'],
        "my_table",
        ['col1'],
        ['col2']
    ) ->
    INSERT INTO my_table (col1, col2, col3, col4) VALUES %s
    ON CONFLICT (col1) DO UPDATE SET (col3, col4) = (EXCLUDED.col3, EXCLUDED.col4) ;
    :param cols: the postgres table columns required in the
        insert part of the query.
    :param table_name: the postgres table name.
    :param unique_key: unique_key of the postgres table for checking
        unique constraint violations.
    :param cols_not_for_update: columns in cols which are not required in
        the update part of upsert query.
    :return: Upsert query as per input arguments.
    """
    cols = [f'"{col}"' for col in cols]
    cols_str = ', '.join(cols)
    insert_query = """ INSERT INTO %s (%s) VALUES %%s """ % (
        table_name, cols_str
    )
    if cols_not_for_update is not None:
        cols_not_for_update.extend(unique_key)
    else:
        cols_not_for_update = [col for col in unique_key]
    cols_not_for_update = [f'"{col}"' for col in cols_not_for_update]
    unique_key = [f'"{col}"' for col in unique_key]
    unique_key_str = ', '.join(unique_key)

    update_cols = [f"{col}" for col in cols if col not in cols_not_for_update]
    update_cols_str = ', '.join(update_cols)
    update_cols_with_excluded_markers = [f'EXCLUDED.{col}' for col in update_cols]
    update_cols_with_excluded_markers_str = ', '.join(
        update_cols_with_excluded_markers
    )
    if len(update_cols) > 1:
        equality_clause = "(%s) = (%s)"
    else:
        equality_clause = "%s = %s"

    on_conflict_clause = f""" ON CONFLICT (%s) DO UPDATE SET {equality_clause} ;"""
    on_conflict_clause = on_conflict_clause % (
        unique_key_str,
        update_cols_str,
        update_cols_with_excluded_markers_str
    )
    if len(unique_key) == 0:
        return insert_query
    return insert_query + on_conflict_clause

In [4]:
storage_client = storage.Client(project="ytone-430507")
today = datetime.now()
table_name = "douyin_price"
api_name = "handler_post"
bucket_name = "3_staging_area"

In [5]:
processing_blobs = [
{
    "blob": blob,
    "date": blob.name.split('/')[2],
    "batch": int(blob.name.split('/')[-1].replace(".parquet", "").split("_")[-1]),
} for blob in storage_client.list_blobs("3_staging_area",prefix="1_xingtu/" + table_name + "/") if api_name in blob.name]
processing_blobs

[{'blob': <Blob: 3_staging_area, 1_xingtu/douyin_price/2024-09-13/handler_post_241009_0.parquet, 1728469231343006>,
  'date': '2024-09-13',
  'batch': 0},
 {'blob': <Blob: 3_staging_area, 1_xingtu/douyin_price/2024-09-13/handler_post_241009_1.parquet, 1728469233446157>,
  'date': '2024-09-13',
  'batch': 1},
 {'blob': <Blob: 3_staging_area, 1_xingtu/douyin_price/2024-10-02/handler_post_241009_0.parquet, 1728469234776362>,
  'date': '2024-10-02',
  'batch': 0},
 {'blob': <Blob: 3_staging_area, 1_xingtu/douyin_price/2024-10-02/handler_post_241009_1.parquet, 1728469236516667>,
  'date': '2024-10-02',
  'batch': 1},
 {'blob': <Blob: 3_staging_area, 1_xingtu/douyin_price/2024-10-02/handler_post_241009_2.parquet, 1728469238772536>,
  'date': '2024-10-02',
  'batch': 2},
 {'blob': <Blob: 3_staging_area, 1_xingtu/douyin_price/2024-10-02/handler_post_241009_3.parquet, 1728469242338224>,
  'date': '2024-10-02',
  'batch': 3},
 {'blob': <Blob: 3_staging_area, 1_xingtu/douyin_price/2024-10-02/hand

In [6]:
bucket = storage_client.get_bucket(bucket_name)
meta_blob = bucket.blob("1_xingtu/" + table_name + "/meta.json")
if meta_blob.exists():
    processed_blobs = json.loads(meta_blob.download_as_string())
else:
    processed_blobs = []
processed_blobs


[{'file_path': 'gs://3_staging_area/1_xingtu/douyin_price/handler_post_241009_0.parquet',
  'date': '2024-09-13',
  'batch': 0},
 {'file_path': 'gs://3_staging_area/1_xingtu/douyin_price/handler_post_241009_1.parquet',
  'date': '2024-09-13',
  'batch': 1},
 {'file_path': 'gs://3_staging_area/1_xingtu/douyin_price/handler_post_241009_0.parquet',
  'date': '2024-10-02',
  'batch': 0},
 {'file_path': 'gs://3_staging_area/1_xingtu/douyin_price/handler_post_241009_1.parquet',
  'date': '2024-10-02',
  'batch': 1},
 {'file_path': 'gs://3_staging_area/1_xingtu/douyin_price/handler_post_241009_2.parquet',
  'date': '2024-10-02',
  'batch': 2},
 {'file_path': 'gs://3_staging_area/1_xingtu/douyin_price/handler_post_241009_3.parquet',
  'date': '2024-10-02',
  'batch': 3},
 {'file_path': 'gs://3_staging_area/1_xingtu/douyin_price/handler_post_241009_4.parquet',
  'date': '2024-10-02',
  'batch': 4},
 {'file_path': 'gs://3_staging_area/1_xingtu/douyin_price/handler_post_241009_5.parquet',
  'date

In [7]:
to_process = []
for processing_blob in processing_blobs:
    processing_date = datetime.strptime(processing_blob["date"], "%Y-%m-%d")
    if processing_date >= datetime(2024, 9, 8):
        if processing_blob["batch"] not in [processed_blob["batch"] for processed_blob in processed_blobs if api_name in processed_blob["file_path"] and processing_blob["date"] == processed_blob["date"]]:
            to_process.append(processing_blob)
pprint(to_process)
print(len(to_process))

[{'batch': 0,
  'blob': <Blob: 3_staging_area, 1_xingtu/douyin_price/2024-10-11/handler_post_241011_0.parquet, 1728637234035152>,
  'date': '2024-10-11'},
 {'batch': 1,
  'blob': <Blob: 3_staging_area, 1_xingtu/douyin_price/2024-10-11/handler_post_241011_1.parquet, 1728637237300243>,
  'date': '2024-10-11'},
 {'batch': 2,
  'blob': <Blob: 3_staging_area, 1_xingtu/douyin_price/2024-10-11/handler_post_241011_2.parquet, 1728637239002546>,
  'date': '2024-10-11'}]
3


In [8]:
engine = create_engine("postgresql+psycopg2://{user}:{password}@{host}:{port}/{database}".format(**postgres_config))
m_df = pd.read_sql_table("douyin_influencer",con=engine)
for item in to_process:
    l_df = pd.read_sql_table("douyin_price",con=engine)
    r_df = pd.read_parquet("gs://" + bucket_name + "/" + item["blob"].name)
    df = pd.merge(m_df, r_df, left_on="user_id", right_on="influencer_id", how="inner")
    df = df[["id"] + [column for column in r_df.columns if "influencer_id" not in column]]
    upsert_df = df.rename(columns={"id": "influencer_id"})
    upsert_df["has_discount"] = False
    query = build_upsert_query(upsert_df.columns, "douyin_price", ["influencer_id", "video_type"])
    value = ", ".join([str(record).replace("''", "NULL") for record in upsert_df.fillna("").to_records(index=False)])
    query = query % value
    with engine.connect() as conn:
        result = conn.execute(text(query))
        conn.commit()
    processed_blobs.append({
        'file_path': "gs://" + bucket_name + "/1_xingtu/" + table_name + "/" + api_name + "_" + today.strftime("%y%m%d") + "_" + str(item["batch"]) + ".parquet",
        'date': item["date"],
        'batch': item["batch"]
    })
    meta_blob.upload_from_string(json.dumps(processed_blobs))